In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 3

In [3]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_382681/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_382681/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.5907332130840847,
 'decoded_ood_sinkhorn_div_1': 119.333493913923,
 'decoded_ood_sinkhorn_div_10': 86.693605695452,
 'decoded_ood_sinkhorn_div_100': 41.08837563650949,
 'decoded_ood_e_distance': 81.15640115354623,
 'decoded_ood_mmd': 0.41219520568847656}

In [14]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.766290195286274,
 'decoded_test_e_distance': 45.028527655774745,
 'decoded_test_mmd_distance': 0.38532132655382156}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.5907332130840847,
 'decoded_ood_sinkhorn_div_1': 119.333493913923,
 'decoded_ood_sinkhorn_div_10': 86.693605695452,
 'decoded_ood_sinkhorn_div_100': 41.08837563650949,
 'decoded_ood_e_distance': 81.15640115354623,
 'decoded_ood_mmd': 0.41219520568847656}

In [16]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -13.23927243266787,
 'encoded_ood_sinkhorn_div_1': 39.20618438720703,
 'encoded_ood_sinkhorn_div_10': 34.86188357216971,
 'encoded_ood_sinkhorn_div_100': 33.329532623291016,
 'encoded_ood_e_distance': 66.29000940015871,
 'encoded_ood_mmd': 0.4421341674668448}

In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.26182221514838083,
 'deg_ood_sinkhorn_div_1': 33.75623457772391,
 'deg_ood_sinkhorn_div_10': 24.610683577401296,
 'deg_ood_sinkhorn_div_100': 23.833501952035085,
 'deg_ood_e_distance': 47.505811492812384,
 'deg_ood_mmd': 0.47036358288356234}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.6033340469002724,
 'deg_test_sinkhorn_div_1': 22.144510090351105,
 'deg_test_sinkhorn_div_10': 13.681477705637613,
 'deg_test_sinkhorn_div_100': 12.637494186560312,
 'deg_test_e_distance': 25.127830429197445,
 'deg_test_mmd': 0.4053037613630295}

In [19]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.766290195286274,
 'decoded_test_e_distance': 45.028527655774745,
 'decoded_test_mmd_distance': 0.38532132655382156}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.5907332130840847,
 'decoded_ood_sinkhorn_div_1': 119.333493913923,
 'decoded_ood_sinkhorn_div_10': 86.693605695452,
 'decoded_ood_sinkhorn_div_100': 41.08837563650949,
 'decoded_ood_e_distance': 81.15640115354623,
 'decoded_ood_mmd': 0.41219520568847656}

In [21]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.26182221514838083,
 'deg_ood_sinkhorn_div_1': 33.75623457772391,
 'deg_ood_sinkhorn_div_10': 24.610683577401296,
 'deg_ood_sinkhorn_div_100': 23.833501952035085,
 'deg_ood_e_distance': 47.505811492812384,
 'deg_ood_mmd': 0.47036358288356234}

In [22]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.766290195286274,
 'decoded_test_e_distance': 45.028527655774745,
 'decoded_test_mmd_distance': 0.38532132655382156}

In [23]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.28669184911693457,
 'encoded_test_sinkhorn_div_1': 26.625403960545857,
 'encoded_test_sinkhorn_div_10': 22.098695238431294,
 'encoded_test_sinkhorn_div_100': 20.72038479646047,
 'encoded_test_e_distance': 41.15412285976735,
 'encoded_test_mmd': 0.3527084638675054}

In [24]:
ood_metrics_encoded

{'Panobinostat+Crizotinib': {'r_squared': -7.4062347412109375,
  'sinkhorn_div_1': 73.20408630371094,
  'sinkhorn_div_10': 66.83192443847656,
  'sinkhorn_div_100': 64.30015563964844,
  'e_distance': 127.95356212063142,
  'mmd': 0.55925006},
 'Panobinostat+Curcumin': {'r_squared': -19.73055076599121,
  'sinkhorn_div_1': 54.90412139892578,
  'sinkhorn_div_10': 49.01411819458008,
  'sinkhorn_div_100': 46.95843505859375,
  'e_distance': 93.43057892695266,
  'mmd': 0.5076148},
 'Panobinostat+SRT1720': {'r_squared': -53.114505767822266,
  'sinkhorn_div_1': 47.72954559326172,
  'sinkhorn_div_10': 42.042442321777344,
  'sinkhorn_div_100': 40.00395202636719,
  'e_distance': 79.54415512165181,
  'mmd': 0.49887857},
 'Panobinostat+Sorafenib': {'r_squared': -13.904767036437988,
  'sinkhorn_div_1': 63.86777877807617,
  'sinkhorn_div_10': 57.541259765625,
  'sinkhorn_div_100': 55.18510437011719,
  'e_distance': 109.80701336320027,
  'mmd': 0.5311838},
 'SRT2104+Alvespimycin': {'r_squared': 0.8950974

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/chemcpa"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
